#### Imports

In [12]:
import PyPDF2
import csv

#### Enter info about the pdf

In [13]:
pdfFileObj = open('directory_2017.pdf', 'rb')
pdfReader = PyPDF2.PdfFileReader(pdfFileObj)
print("Number of pages:", pdfReader.numPages)

Number of pages: 1422


#### Scrape key to facility codes
- 2 problems to solve if accurate key info is needed (right now just using the list of abbr.): 
    - titles of sections?
    - lines for value are sometimes splitted thus not included

In [14]:
abbr_key = {}
def clean_key_page(raw_abbr_key):
    raw_abbr_key[:] = [l for l in raw_abbr_key if (l.strip()!= '')]
    i = 0
    while i < len(raw_abbr_key):
        #print(raw_abbr_key[i])
        if raw_abbr_key[i].isupper():
            abbr_key[raw_abbr_key[i].strip()] = raw_abbr_key[i+1].strip()
            i += 2
        else:
            spitted = raw_abbr_key[i].split()
            if spitted[0].isupper():
                abbr_key[spitted[0]] = ', '.join(spitted[1:])
            i += 1

In [15]:
raw_abbr_key1 = pdfReader.getPage(4).extractText()
raw_abbr_key1 = raw_abbr_key1.split('\n')
raw_abbr_key1 = raw_abbr_key1[1:len(raw_abbr_key1) - 4]
clean_key_page(raw_abbr_key1)

In [16]:
raw_abbr_key2 = pdfReader.getPage(5).extractText()
raw_abbr_key2 = raw_abbr_key2.split('\n')
raw_abbr_key2 = raw_abbr_key2[:len(raw_abbr_key2) - 4]
clean_key_page(raw_abbr_key2)

In [17]:
raw_abbr_key3 = pdfReader.getPage(6).extractText()
raw_abbr_key3 = raw_abbr_key3.split('\n')
raw_abbr_key3 = raw_abbr_key3[26:len(raw_abbr_key3)-1]
clean_key_page(raw_abbr_key3)

In [18]:
abbr_key

{'GHF': 'General Health Services',
 'MHF': 'Mental Health Treatment Services',
 'MHSAF': 'Mix of Mental Health and Substance Abuse',
 'SAF': 'Substance Abuse Treatment Services',
 'ACM': 'Acamprosate (Campral®)',
 'BM': 'Buprenorphine, maintenance',
 'BMW': 'Buprenorphine, maintenance, for, a, pre-',
 'BU': 'Buprenorphine, used, in, treatment',
 'DB': 'Buprenorphine, detoxi˝cation',
 'DM': 'Methadone, detoxi˝cation',
 'DSF': 'Disul˝ram (Antabuse®)',
 'DT': 'Detoxi˝cation',
 'HH': 'Transitional, housing, or, halfway, house',
 'METH': 'Methadone',
 'MM': 'Methadone maintenance',
 'MMW': 'Methadone maintenance for pre-determined',
 'MOA': 'Accepts clients on opioid medication',
 'MPD': 'Medications for psychiatric disorders',
 'NMOA': 'Does not use medication for opioid addiction',
 'NOOP': 'Does not treat opioid addiction',
 'NXN': 'Naltrexone, (oral)',
 'OTP': 'SAMHSA-certi˝ed Opioid Treatment Program',
 'OTPA': 'All clients in Opioid Treatment Program',
 'PAIN': 'Use methadone/buprenor

#### Scraper test on a page of data
- data starts on page 15 (getPage(14))

In [19]:
page = pdfReader.getPage(15).extractText()
rawdata = page.split('\n')
rawdata

['',
 'KEY',
 'Northwest Alabama Treatment Center',
 '4204 Edmonton Drive',
 'Bessemer, AL 35022',
 '(205) 425-1200',
 'SAF ',
 ' SA DT MM MMW DM METH ',
 'OTPA OTP ',
 ' CBT DBT SACA TRC ',
 'REBT TWFA BIA CMI MOTI ANG ',
 'MXM RELP ',
 ' SMPD ',
 ' OP OMB ORT ',
 ' PVT ',
 ' STAG STMH CARF ',
 ' SF MD ',
 ' CO GL VET ADM MF CJ SE PW WN ',
 'MN HV TRMA XA DV TAY ',
 ' CM PEER ',
 'HEOH HS SSA SMHD CSAA CMHA ',
 'OPC ISC DAUT STDT TBS DP ACC SSD ',
 'AOSS EMP DVFP EIH HAEC TAEC ',
 'SAE MHS SHG ICO GCO FCO MCO ',
 ' YAD ADLT ',
 ' FEM MALE ',
 ' MOAlcohol and Drug Abuse',
 'Treatment Centers Inc/Pearson Hall',
 '2701 Je˜erson Avenue SW',
 'Birmingham, AL 35211',
 '(205) 923-6552x12',
 'Intakes: (205) 923-6552x10 (205) 923-',
 '6552x13',
 'SAF ',
 ' SA MPD NMOA ',
 ' SACA TRC ',
 'TWFA BIA MOTI ANG RELP ',
 ' SMPD ',
 ' RES RS ',
 ' PVT ',
 ' STAG STMH STDH ',
 ' SF FSA ',
 ' SS PA ',
 ' HS SSA SMHD CSAA ',
 'OPC ISC BABA DAUT SHB SHC HIVT ',
 'STDT TBS DP ACC EIH HAEC SAE ',
 'SHG ICO 

In [20]:
def clean_data(rawdata):
    '''
    Roughly split data, returns list of list.
    Inputs:
        rawdata_from_page(list): list of items on the page that were seperated by /n
    '''
    #delete na lines
    rawdata[:] = [l.strip() for l in rawdata if (l != '' and l != 'KEY')]
    #split erroneously connected data
    data = []
    temp = []
    i = 0
    while i < len(rawdata) - 1:
        if rawdata[i].isupper() and (not rawdata[i+1].isupper()): #if i is the last line of keys, i+1 is facility name
            temp.append(rawdata[i])
            data.append(temp)
            temp = []
        else:
            temp.append(rawdata[i])
        i += 1
    temp.append(rawdata[i])
    data.append(temp)
    return data


def clean_facility_name(fname):
    '''
    Split facility name into ["junk", actual name]
    fname(str): name of the facility
    '''
    front = ''
    i = 0
    while i < len(fname) - 1:
        if fname[i].isupper() and fname[i+1].isupper():
            front += fname[i]
            i += 1
        else:
            break
    if front in abbr_key:
        return [front, fname[i:]]
    else:
        return ['', fname[i:]]
    
    
def clean_all_facility_name(data):
    '''
    Move and link wrongly splitted data, changes in place
    '''
    i = 0
    while i < len(data):
        facility_name_raw = data[i][0]
        if facility_name_raw[:2].isupper():
            code, fac_name = clean_facility_name(facility_name_raw)
            if i != 0:
                data[i][0] = fac_name
                data[i - 1].append(code)
        i += 1

In [21]:
k = clean_data(rawdata)

In [22]:
k

[['Northwest Alabama Treatment Center',
  '4204 Edmonton Drive',
  'Bessemer, AL 35022',
  '(205) 425-1200',
  'SAF',
  'SA DT MM MMW DM METH',
  'OTPA OTP',
  'CBT DBT SACA TRC',
  'REBT TWFA BIA CMI MOTI ANG',
  'MXM RELP',
  'SMPD',
  'OP OMB ORT',
  'PVT',
  'STAG STMH CARF',
  'SF MD',
  'CO GL VET ADM MF CJ SE PW WN',
  'MN HV TRMA XA DV TAY',
  'CM PEER',
  'HEOH HS SSA SMHD CSAA CMHA',
  'OPC ISC DAUT STDT TBS DP ACC SSD',
  'AOSS EMP DVFP EIH HAEC TAEC',
  'SAE MHS SHG ICO GCO FCO MCO',
  'YAD ADLT',
  'FEM MALE'],
 ['MOAlcohol and Drug Abuse',
  'Treatment Centers Inc/Pearson Hall',
  '2701 Je˜erson Avenue SW',
  'Birmingham, AL 35211',
  '(205) 923-6552x12',
  'Intakes: (205) 923-6552x10 (205) 923-',
  '6552x13',
  'SAF',
  'SA MPD NMOA',
  'SACA TRC',
  'TWFA BIA MOTI ANG RELP',
  'SMPD',
  'RES RS',
  'PVT',
  'STAG STMH STDH',
  'SF FSA',
  'SS PA',
  'HS SSA SMHD CSAA',
  'OPC ISC BABA DAUT SHB SHC HIVT',
  'STDT TBS DP ACC EIH HAEC SAE',
  'SHG ICO GCO FCO MCO',
  'YAD'

In [23]:
clean_all_facility_name(k)

In [24]:
k

[['Northwest Alabama Treatment Center',
  '4204 Edmonton Drive',
  'Bessemer, AL 35022',
  '(205) 425-1200',
  'SAF',
  'SA DT MM MMW DM METH',
  'OTPA OTP',
  'CBT DBT SACA TRC',
  'REBT TWFA BIA CMI MOTI ANG',
  'MXM RELP',
  'SMPD',
  'OP OMB ORT',
  'PVT',
  'STAG STMH CARF',
  'SF MD',
  'CO GL VET ADM MF CJ SE PW WN',
  'MN HV TRMA XA DV TAY',
  'CM PEER',
  'HEOH HS SSA SMHD CSAA CMHA',
  'OPC ISC DAUT STDT TBS DP ACC SSD',
  'AOSS EMP DVFP EIH HAEC TAEC',
  'SAE MHS SHG ICO GCO FCO MCO',
  'YAD ADLT',
  'FEM MALE',
  'MO'],
 ['Alcohol and Drug Abuse',
  'Treatment Centers Inc/Pearson Hall',
  '2701 Je˜erson Avenue SW',
  'Birmingham, AL 35211',
  '(205) 923-6552x12',
  'Intakes: (205) 923-6552x10 (205) 923-',
  '6552x13',
  'SAF',
  'SA MPD NMOA',
  'SACA TRC',
  'TWFA BIA MOTI ANG RELP',
  'SMPD',
  'RES RS',
  'PVT',
  'STAG STMH STDH',
  'SF FSA',
  'SS PA',
  'HS SSA SMHD CSAA',
  'OPC ISC BABA DAUT SHB SHC HIVT',
  'STDT TBS DP ACC EIH HAEC SAE',
  'SHG ICO GCO FCO MCO',
 

In [40]:
def write_data(data):
    '''
    write data into right columns
    '''
    with open('2017.csv', 'w', newline='') as file:
        writer = csv.writer(file)
        writer.writerow(["Center_name", "Street_address", "City", "State", "Postal_code", "Phone", "Keys"])
        for dc in data:
            i = 0
            data_holder = []
            while i < len(dc):
                if dc[i][0].isnumeric() and (len(data_holder) == 0): #if ith place has the street address
                    data_holder.append(' '.join(dc[:i-1])) #everything before is the name of the center
                    i += 1
                    address_info = dc[i].split()
                    address_info[0] = address_info[0][:len(address_info[0]) - 1]
                    data_holder += address_info
                    i += 1
                    break
                i += 1
            j = i
            while j < len(dc):
                if dc[j].isupper():
                    break
                j += 1
            data_holder.append(' '.join(dc[i:j]))
            data_holder.append(' '.join(dc[j:]))
            writer.writerow(data_holder)     

In [41]:
write_data(k)